In [1]:
import os
import csv, subprocess
import sys
import pandas as pd
import itertools
from tqdm import tqdm
from functools import reduce

In [2]:
cohort_id='bellenguez_omics_dl'

remove_cols = ['microglia_enhancer_intersect_atac',
               'microglia-enhancer_promoter_union_atac-microglia-zscore_1_zscore_1',
               'microglia-enhancer_promoter_union_atac-microglia-zscore_2_zscore_2',
               #'microglia-enhancer_promoter_union_atac-microglia-zscore_3_zscore_3',
               'neuron-enhancer_promoter_union_atac-neuron-zscore_3_zscore_3'
               ]

enformer_cols_subset_1 = ['diff_32_ENCFF748RWG_zscore_1',
                          'diff_32_ENCFF472KMT_zscore_3',
                          'diff_32_ENCFF991YWM_zscore_3',
                          'diff_32_ENCFF476YVH_zscore_2',
                          'diff_32_ENCFF890NAY_zscore_2',
                          'diff_32_ENCFF316ZVD_zscore_1',
                          'diff_32_ENCFF076XUR_zscore_3',
                          'diff_32_ENCFF572IZL_zscore_2'
                          ]

#enformer_cols_subset_1 = ['diff_32_ENCFF714QQU_zscore_2',
#                 'diff_32_ENCFF486BJL_zscore_3',
#                 'diff_32_ENCFF176WTT_zscore_3',
#                 'diff_32_ENCFF520JQX_zscore_1',
#                 'diff_32_ENCFF683KJQ_zscore_3',
#                 'diff_32_ENCFF812BTH_zscore_1',
#                 'diff_32_ENCFF476YVH_zscore_2',
#                 'diff_32_ENCFF720XZQ_zscore_3',
#                 'diff_32_ENCFF316ZVD_zscore_1',
#                 'diff_32_ENCFF945VIU_zscore_1',
#                 'diff_32_ENCFF572IZL_zscore_1',
#                 'diff_32_ENCFF748RWG_zscore_1',
#                 'diff_32_ENCFF991YWM_zscore_3',
#                 'diff_32_ENCFF890NAY_zscore_2',
#                 'diff_32_ENCFF845QWZ_zscore_3',
#                 'diff_32_ENCFF076XUR_zscore_3',
#                 'diff_32_ENCFF472KMT_zscore_3']

meta_cols = ['CHR', 'SNP', 'BP', 'A1', 'A2']

bl='/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/annotations_hg38/merged_annotations_ADSP_v2/baseline_filtered/baseline_chr'
glass_lab='/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/annotations_hg38/merged_annotations_ADSP_v2/annotations_robust/brain_all/brain_all_chr'
roadmap='/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/annotations_hg38/merged_annotations_ADSP_v2/annotations_robust/roadmap/roadmap_chr'
enformer='/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/annotations_hg38/merged_annotations_ADSP_v2/annotations_robust/enformer/enformer_chr'
glass_lab_enformer='/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/annotations_hg38/merged_annotations_ADSP_v2/annotations_robust/enformer_glass_lab/enformer_tensorflow_TF_glass_chr'
path_sumstats='/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/summary_stats/alzheimers/analysis_05_05_2024/original_sumstats/flipped/Bellenguez_et_al_2021_stage1_hg38_flipped_munged_MAF_v2.parquet'
weight='/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/annotations_hg38/merged_annotations_ADSP_v2/weights/weights_chr'
path_out=f'/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/summary_stats/alzheimers/ADSP_reference_panel/fine_mapping/annotations_dl/{cohort_id}'
file_out=f'{path_out}/{cohort_id}'


## check anno
- Baseline,roadmap are all the same
- There's only one from glasslab
- Enformer only took those from Enformer_subset1
- Some were removed in Enformer_glasslab too

In [15]:
bl_file= f'{bl}22.annot.gz'
bl_string = pd.read_csv(bl_file, sep='\t').columns.tolist()
bl_string_new = [col for col in bl_string if col not in meta_cols]
print([cols for cols in bl_string if cols not in bl_string_new])

['CHR', 'SNP', 'BP', 'A1', 'A2']


In [31]:
glass_file= f'{glass_lab}22.annot.gz'
glass_string = pd.read_csv(glass_file, sep='\t').columns.tolist()
glass_string_new = [col for col in glass_string if col not in meta_cols]
glass_string_new = [col for col in glass_string_new  if col not in remove_cols ]
print([cols for cols in glass_string if cols not in glass_string_new])

['CHR', 'SNP', 'BP', 'A1', 'A2', 'microglia_enhancer_intersect_atac']


In [19]:
roadmap_file= f'{roadmap}22.annot.gz'
roadmap_string = pd.read_csv(roadmap_file, sep='\t').columns.tolist()
roadmap_string_new = [col for col in roadmap_string if col not in meta_cols]
print([cols for cols in roadmap_string if cols not in roadmap_string_new])

['CHR', 'SNP', 'BP', 'A1', 'A2']


In [21]:
enformer_file= f'{enformer}22.annot.gz'
enformer_string = pd.read_csv(enformer_file, sep='\t').columns.tolist()
enformer_string_new = [col for col in enformer_string if col not in meta_cols]
enformer_string_new = [col for col in enformer_string if col in enformer_cols_subset_1]
print([cols for cols in enformer_string if cols not in enformer_string_new])

['CHR', 'SNP', 'BP', 'A1', 'A2', 'diff_32_ENCFF714QQU_zscore_2', 'diff_32_ENCFF486BJL_zscore_3', 'diff_32_ENCFF176WTT_zscore_3', 'diff_32_ENCFF520JQX_zscore_1', 'diff_32_ENCFF520JQX_zscore_2', 'diff_32_ENCFF683KJQ_zscore_3', 'diff_32_ENCFF812BTH_zscore_1', 'diff_32_ENCFF720XZQ_zscore_3', 'diff_32_ENCFF945VIU_zscore_1', 'diff_32_ENCFF572IZL_zscore_1', 'diff_32_ENCFF845QWZ_zscore_3']


In [38]:
enformer_glass_file = f'{glass_lab_enformer}22.annot.gz'
enformer_glass_string = pd.read_csv(enformer_glass_file, sep='\t').columns.tolist()
enformer_glass_string_new = [col for col in enformer_glass_string if col not in meta_cols]
enformer_glass_string_new = [col for col in enformer_glass_string_new  if col not in remove_cols]

print([cols for cols in enformer_glass_string if cols not in enformer_glass_string_new])

['CHR', 'SNP', 'BP', 'A1', 'A2', 'microglia-enhancer_promoter_union_atac-microglia-zscore_1_zscore_1', 'microglia-enhancer_promoter_union_atac-microglia-zscore_2_zscore_2', 'neuron-enhancer_promoter_union_atac-neuron-zscore_3_zscore_3']


In [63]:
annot_string_array = bl_string_new + glass_string_new + roadmap_string_new + enformer_string_new + enformer_glass_string_new
len(annot_string_array)


96

## get annotation for omics, omics_dl

In [55]:
bl_path='/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/annotations_hg38/merged_annotations_ADSP_v2/baseline_filtered/baseline_chr'
glasslab_path='/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/annotations_hg38/merged_annotations_ADSP_v2/annotations_robust/brain_all//brain_all_chr'
roadmap_path='/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/annotations_hg38/merged_annotations_ADSP_v2/roadmap/roadmap_chr'

enformer_path='/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/annotations_hg38/merged_annotations_ADSP_v2/annotations_robust/enformer/enformer_chr'
enformer_glasslab_path='/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/annotations_hg38/merged_annotations_ADSP_v2/annotations_robust/enformer_glass_lab/enformer_tensorflow_TF_glass_chr'

In [75]:
 for i in tqdm(range(1,21)):

    bl = pd.read_csv(bl_path+str(i)+'.annot.gz',compression='gzip', sep = '\t', usecols= meta_cols + bl_string_new)
    roadmap = pd.read_csv(roadmap_path+str(i)+'.annot.gz',compression='gzip', sep = '\t', usecols= meta_cols + roadmap_string_new )
    glasslab = pd.read_csv(glasslab_path+str(i)+'.annot.gz',compression='gzip', sep = '\t', usecols= meta_cols + glass_string_new)

    enformer= pd.read_csv(enformer_path+str(i)+'.annot.gz',compression='gzip', sep = '\t', usecols= meta_cols + enformer_string_new)
    enformer_glasslab = pd.read_csv(enformer_glasslab_path+str(i)+'.annot.gz',compression='gzip', sep = '\t', usecols= meta_cols + enformer_glass_string_new)


    dfs = [bl, roadmap, glasslab]
    omics = reduce(lambda left, right: pd.merge(left, right, on=meta_cols), dfs)

    dfs = [omics, enformer, enformer_glasslab]
    omics_dl = reduce(lambda left, right: pd.merge(left, right, on=meta_cols), dfs)

    # write to files
    omics.to_csv('/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/summary_stats/alzheimers/ADSP_reference_panel/fine_mapping/annotations_dl/teresa/anno_dec/omics_chr'+str(i) +'.annot.gz',
                   compression='gzip', sep = '\t', index = False)

    omics_dl.to_csv('/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/summary_stats/alzheimers/ADSP_reference_panel/fine_mapping/annotations_dl/teresa/anno_dec/omics_dl_chr'+str(i) +'.annot.gz',
                   compression='gzip', sep = '\t', index = False)

100%|██████████| 20/20 [35:36<00:00, 106.83s/it] 


In [68]:
omics_dl

,CHR,SNP,BP,A1,A2,Coding_UCSC,UTR_5_UCSC,Repressed_Hoffman.extend.500,H3K4me1_peaks_Trynka,GTEx_FE_META_TISSUE_GE_MaxCPP,...,diff_32_ENCFF316ZVD_zscore_1,diff_32_ENCFF572IZL_zscore_2,diff_32_ENCFF748RWG_zscore_1,diff_32_ENCFF991YWM_zscore_3,diff_32_ENCFF890NAY_zscore_2,diff_32_ENCFF076XUR_zscore_3,diff_32_ENCFF472KMT_zscore_3,microglia-enhancer_promoter_union_atac-microglia-zscore_3_zscore_3,microglia-enhancer_promoter_union_atac_500-microglia-zscore_1_zscore_1,neuron-enhancer_promoter_union_atac_500-neuron-zscore_3_zscore_3
0,22,rs1302460385,10515914,G,A,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,22,rs1192072962,10515927,G,A,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,22,rs1262543443,10515938,A,G,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,22,rs1404924098,10516018,A,G,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,22,rs1163591173,10516212,T,G,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336055,22,rs1279358620,50807604,G,C,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
336056,22,rs1486437805,50807639,A,G,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
336057,22,rs1381773798,50807699,G,T,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
336058,22,rs1250543512,50807702,G,A,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:

annot_string_array = bl_string + glass_string + roadmap_string + enformer_string + enformer_glass_string
#annot_string_array = bl_string + glass_string + roadmap_string


annot_string_array = [col for col in annot_string_array if col not in remove_cols]

annot_string = ','.join(annot_string_array)

annot_files = ','.join([bl, glass_lab, roadmap, enformer, glass_lab_enformer])


#h2: 0.051
submit_command = (f'python polyfun.py ' 
                  f'--compute-h2-L2 '
                  f'--no-partitions '
                  f'--output-prefix {file_out} '
                  f'--sumstats {path_sumstats} '
                  f'--ref-ld-chr {annot_files} '
                  f'--w-ld-chr {weight} '
                  f'--allow-missing '
                  f'--nnls-exact '
                  f'--anno {annot_string} '
                  f'--q 100000'
                  )

 #--print-delete-vals


print(submit_command + '\n' + '\n')  # Uncomment this line when done testing to use the submit command created

